In [3]:
pip install nsepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from nsepy import get_history
from datetime import datetime, timedelta


In [5]:
top_stocks = [
    'RELIANCE', 'TCS', 'HDFC BANK', 'INFOSYS', 'HINDUSTAN UNILEVER',
    'ICICI BANK', 'KOTAK BANK', 'LT', 'STATE BANK OF INDIA', 'ITC',
    'BAJAJ FINANCE', 'ASIAN PAINTS', 'HDFC', 'TITAN', 'M&M',
    'ULTRATECH CEMENT', 'WIPRO', 'TECH MAHINDRA', 'NTPC', 'POWER GRID'
]


In [6]:
# Define the date range
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

# Create a dictionary to store DataFrames
data_dict = {}

for stock in top_stocks:
    data = get_history(symbol=stock, start=start_date, end=end_date)
    data_dict[stock] = data[['Open', 'High', 'Low', 'Close', 'Volume']]


In [11]:
import numpy as np

def moving_average_crossover(data, short_window=20, long_window=50):
    signals = pd.DataFrame(index=data.index)
    signals['price'] = data['Close']

    # Calculate short and long moving averages
    signals['short_mavg'] = data['Close'].rolling(window=short_window, min_periods=1).mean()
    signals['long_mavg'] = data['Close'].rolling(window=long_window, min_periods=1).mean()

    # Create signals
    signals['signal'] = 0
    # Set signal to 1 where short_mavg > long_mavg
    signals['signal'] = np.where(signals['short_mavg'] > signals['long_mavg'], 1, 0)

    # Generate trading orders
    signals['positions'] = signals['signal'].diff()

    return signals

In [12]:
def backtest_strategy(data, signals):
    initial_capital = 100000.0
    shares = 10  # Number of shares to buy/sell
    portfolio = pd.DataFrame(index=signals.index)
    portfolio['holdings'] = signals['positions'].cumsum() * data['Close']
    portfolio['cash'] = initial_capital - (signals['positions'] * shares * data['Close']).cumsum()
    portfolio['total'] = portfolio['cash'] + portfolio['holdings']

    return portfolio

In [13]:
def calculate_performance(portfolio):
    total_return = (portfolio['total'][-1] - portfolio['total'][0]) / portfolio['total'][0]
    annualized_return = (1 + total_return) ** (252 / len(portfolio)) - 1
    volatility = portfolio['total'].pct_change().std() * np.sqrt(252)

    return total_return, annualized_return, volatility

In [15]:
results = {}

for stock in top_stocks:
    if stock in data_dict:
        signals = moving_average_crossover(data_dict[stock])
        portfolio = backtest_strategy(data_dict[stock], signals)

        # Check if portfolio is empty
        if portfolio.empty:
            print(f"Portfolio for {stock} is empty. Skipping performance calculation.")
            continue

        total_return, annualized_return, volatility = calculate_performance(portfolio)

        results[stock] = {
            'Total Return': total_return,
            'Annualized Return': annualized_return,
            'Volatility': volatility
        }

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
print(results_df)

Portfolio for RELIANCE is empty. Skipping performance calculation.
Portfolio for TCS is empty. Skipping performance calculation.
Portfolio for HDFC BANK is empty. Skipping performance calculation.
Portfolio for INFOSYS is empty. Skipping performance calculation.
Portfolio for HINDUSTAN UNILEVER is empty. Skipping performance calculation.
Portfolio for ICICI BANK is empty. Skipping performance calculation.
Portfolio for KOTAK BANK is empty. Skipping performance calculation.
Portfolio for LT is empty. Skipping performance calculation.
Portfolio for STATE BANK OF INDIA is empty. Skipping performance calculation.
Portfolio for ITC is empty. Skipping performance calculation.
Portfolio for BAJAJ FINANCE is empty. Skipping performance calculation.
Portfolio for ASIAN PAINTS is empty. Skipping performance calculation.
Portfolio for HDFC is empty. Skipping performance calculation.
Portfolio for TITAN is empty. Skipping performance calculation.
Portfolio for M&M is empty. Skipping performance ca